-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Orchestrating Jobs with Databricks Workflows

New updates to the Databricks Jobs UI have added the ability to schedule multiple tasks as part of a job, allowing Databricks Jobs to fully handle orchestration for most production workloads.

Here, we'll start by reviewing the steps for scheduling a notebook task as a triggered standalone job, and then add a dependent task using a DLT pipeline. 

## Learning Objectives
By the end of this lesson, you should be able to:
* Schedule a notebook task in a Databricks Workflow Job
* Describe job scheduling options and differences between cluster types
* Review Job Runs to track progress and see results
* Schedule a DLT pipeline task in a Databricks Workflow Job
* Configure linear dependencies between tasks using the Databricks Workflows UI

In [0]:
%run ../../Includes/Classroom-Setup-05.1.1

Resetting the learning environment:
| dropping the schema "hamed_vaheb_jcxq_da_delp_jobs_demo"...(1 seconds)
| removing the working directory "dbfs:/mnt/dbacademy-users/hamed.vaheb@pwc.lu/data-engineer-learning-path/jobs_demo"...(0 seconds)

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v01"

Validating the locally installed datasets:
| listing local files...(1 seconds)
| completed (1 seconds total)

Creating & using the schema "hamed_vaheb_jcxq_da_delp_jobs_demo"...(1 seconds)
Predefined tables in "hamed_vaheb_jcxq_da_delp_jobs_demo":
| -none-

Predefined paths variables:
| DA.paths.working_dir:      dbfs:/mnt/dbacademy-users/hamed.vaheb@pwc.lu/data-engineer-learning-path/jobs_demo
| DA.paths.user_db:          dbfs:/mnt/dbacademy-users/hamed.vaheb@pwc.lu/data-engineer-learning-path/jobs_demo/database.db
| DA.paths.datasets:         dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v01
| DA.paths.storage_location: dbfs:/mnt/db

## Schedule a Notebook Job

When using the Jobs UI to orchestrate a workload with multiple tasks, you'll always begin by scheduling a single task.

Before we start run the following cell to get the values used in this step.

In [0]:
DA.print_job_config_v1()

Job Name:,
Reset Notebook Path:,


Here, we'll start by scheduling the next notebook.

Steps:
1. Click the **Workflows** button on the sidebar.
1. Select the **Jobs** tab.
1. Click the **Create Job** button.
1. Configure the task:
    1. Enter **Reset** for the task name
    1. For **Type**, select **Notebook**
    1. For **Path**, select the **Reset Notebook Path** value provided in the cell above
    1. From the **Cluster** dropdown, under **Existing All Purpose Clusters**, select your cluster
    1. Click **Create**
1. In the top-left of the screen, rename the job (not the task) from **`Reset`** (the defaulted value) to the **Job Name** value provided in the cell above.
1. Click the blue **Run now** button in the top right to start the job.

<img src="https://files.training.databricks.com/images/icon_note_24.png"> **Note**: When selecting your all-purpose cluster, you will get a warning about how this will be billed as all-purpose compute. Production jobs should always be scheduled against new job clusters appropriately sized for the workload, as this is billed at a much lower rate.

In [0]:
DA.validate_job_v1_config()

---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<command-1232511262302884> in <cell line: 1>()
----> 1 DA.validate_job_v1_config()

<command-1232511262302527> in validate_job_v1_config(self)
     19     else:
     20         tasks = settings.get("tasks", [])
---> 21         assert len(tasks) == 1, f"Expected one task, found {len(tasks)}."
     22 
     23         notebook_path = tasks[0].get("notebook_task", {}).get("notebook_path")

AssertionError: Expected one task, found 2.

## Cron Scheduling of Databricks Jobs

Note that on the right hand side of the Jobs UI, directly under the **Job Details** section is a section labeled **Schedule**.

Click on the **Edit schedule** button to explore scheduling options.

Changing the **Schedule type** field from **Manual** to **Scheduled** will bring up a cron scheduling UI.

This UI provides extensive options for setting up chronological scheduling of your Jobs. Settings configured with the UI can also be output in cron syntax, which can be edited if custom configuration not available with the UI is needed.

At this time, we'll leave our job set to the **Manual (Paused)** scheduling type.

## Review Run

As currently configured, our single notebook provides identical performance to the legacy Databricks Jobs UI, which only allowed a single notebook to be scheduled.

To Review the Job Run
1. Select the **Runs** tab in the top-left of the screen (you should currently be on the **Tasks** tab)
1. Find your job. If **the job is still running**, it will be under the **Active runs** section. If **the job finished running**, it will be under the **Completed runs** section
1. Open the Output details by clicking on the timestamp field under the **Start time** column
1. If **the job is still running**, you will see the active state of the notebook with a **Status** of **`Pending`** or **`Running`** in the right side panel. If **the job has completed**, you will see the full execution of the notebook with a **Status** of **`Succeeded`** or **`Failed`** in the right side panel
  
The notebook employs the magic command **`%run`** to call an additional notebook using a relative path. Note that while not covered in this course, <a href="https://docs.databricks.com/repos.html#work-with-non-notebook-files-in-a-databricks-repo" target="_blank">new functionality added to Databricks Repos allows loading Python modules using relative paths</a>.

The actual outcome of the scheduled notebook is to reset the environment for our new job and pipeline.

## Schedule a DLT Pipeline as a Task

The pipeline we create here is a simplified version of the one in the previous unit.

We will use it as part of a scheduled job in this step.

So that we can focus on Jobs and not Pipelines, we are going to use the following utility command to create the pipline for us.

In [0]:
DA.create_pipeline()

Next, we need to add the task to run this pipeline.


Steps:
1. At the top left of your screen, you'll see the **Runs** tab is currently selected; click the **Tasks** tab.
1. Click the large blue circle with a **+** at the center bottom of the screen to add a new task
1. Configure the task:
    1. Enter **DLT** for the task name
    1. For **Type**, select  **Delta Live Tables pipeline**
    1. For **Pipeline**, select the DLT pipeline you configured previously in this exercise<br/>
    Note: The pipeline will start with **DLT-Job-Demo-61** and will end with your email address.
    1. The **Depends on** field defaults to your previously defined task, **Reset** - leave this value as-is.
    1. Click the blue **Create task** button

You should now see a screen with 2 boxes and a downward arrow between them. 

Your **`Reset`** task will be at the top, leading into your **`DLT`** task. 

This visualization represents the dependencies between these tasks.

Click **Run now** to execute your job.

**NOTE**: You may need to wait a few minutes as infrastructure for your job and pipeline is deployed.

In [0]:
DA.validate_job_v2_config()

## Review Multi-Task Run Results

Select the **Runs** tab again and then the most recent run under **Active runs** or **Completed runs** depending on if the job has completed or not.

The visualizations for tasks will update in real time to reflect which tasks are actively running, and will change colors if task failures occur. 

Clicking on a task box will render the scheduled notebook in the UI. 

You can think of this as just an additional layer of orchestration on top of the previous Databricks Jobs UI, if that helps; note that if you have workloads scheduling jobs with the CLI or REST API, <a href="https://docs.databricks.com/dev-tools/api/latest/jobs.html" target="_blank">the JSON structure used to configure and get results about jobs has seen similar updates to the UI</a>.

**NOTE**: At this time, DLT pipelines scheduled as tasks do not directly render results in the Runs GUI; instead, you will be directed back to the DLT Pipeline GUI for the scheduled Pipeline.

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>